In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import joblib

In [35]:
from vincentvanbot.preprocessing import build_pipe_for_categorical
from vincentvanbot.data import create_flat_images_db, get_data_locally, flat_images_db_download, create_joined_img_df
from vincentvanbot.params import IMAGES_PATH, FLAT_IMAGES_DB_PATH_ROOT, BUCKET_NAME, BUCKET_FLAT_IMAGES_DB_FOLDER
from vincentvanbot.data import get_data_locally, flat_images_db_upload, download_images_locally
from tqdm import tqdm
import os

In [4]:
df_transformed = pd.read_pickle("/Users/juliaschmitt/code/JulSchmitt/vincentvanbot/raw_data/flat_resized_images_100.pkl")
df_transformed

,0,1,2,3,4,5,6,7,8,9,...,29990,29991,29992,29993,29994,29995,29996,29997,29998,29999
0,0.180392,0.168627,0.101961,0.262745,0.231373,0.180392,0.243137,0.196078,0.149020,0.184314,...,0.121569,0.149020,0.078431,0.062745,0.164706,0.098039,0.066667,0.149020,0.094118,0.058824
1,0.247059,0.239216,0.188235,0.180392,0.211765,0.160784,0.184314,0.231373,0.192157,0.172549,...,0.188235,0.403922,0.309804,0.207843,0.454902,0.364706,0.243137,0.435294,0.345098,0.223529
10,0.117647,0.086275,0.074510,0.760784,0.772549,0.705882,0.066667,0.050980,0.015686,0.070588,...,0.000000,0.086275,0.015686,0.070588,0.231373,0.149020,0.043137,0.133333,0.039216,0.031373
100,0.211765,0.207843,0.200000,0.219608,0.215686,0.207843,0.200000,0.188235,0.160784,0.223529,...,0.341176,0.419608,0.376471,0.368627,0.474510,0.419608,0.407843,0.431373,0.376471,0.364706
101,0.211765,0.164706,0.109804,0.196078,0.149020,0.094118,0.196078,0.149020,0.094118,0.207843,...,0.109804,0.113725,0.113725,0.105882,0.137255,0.137255,0.129412,0.235294,0.215686,0.192157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.364706,0.313726,0.278431,0.278431,0.235294,0.211765,0.298039,0.270588,0.239216,0.270588,...,0.619608,0.882353,0.839216,0.752941,0.831373,0.788235,0.701961,0.752941,0.705882,0.619608
96,0.443137,0.439216,0.423529,0.482353,0.482353,0.474510,0.396078,0.376471,0.352941,0.403922,...,0.188235,0.443137,0.403922,0.356863,0.486275,0.462745,0.415686,0.466667,0.439216,0.364706
97,0.227451,0.203922,0.203922,0.270588,0.247059,0.247059,0.231373,0.211765,0.196078,0.247059,...,0.568627,0.486275,0.458824,0.427451,0.509804,0.482353,0.450980,0.627451,0.623529,0.603922
98,0.333333,0.341176,0.337255,0.337255,0.337255,0.368627,0.345098,0.364706,0.380392,0.200000,...,0.592157,0.678431,0.674510,0.592157,0.639216,0.654902,0.588235,0.584314,0.627451,0.509804


In [5]:
#X = np.array(df_transformed)

In [6]:
img = np.array(df_transformed.iloc[:1])
img
#img.reshape(100, 100, 3).shape

array([[0.18039216, 0.16862746, 0.10196079, ..., 0.14901961, 0.09411765,
        0.05882353]], dtype=float32)

In [7]:
#knn_model = NearestNeighbors(n_neighbors=20).fit(df_transformed)

In [8]:
ind_list = list(knn_model.kneighbors(img,n_neighbors=20)[1][0])[1:] # get n closest points, unpack indices to a list
df_closest_images = df.iloc[ind_list, :]
df_closest_images.head()

NameError: name 'knn_model' is not defined

In [9]:
df = get_data_locally(100)
df

,AUTHOR,BORN-DIED,TITLE,DATE,TECHNIQUE,LOCATION,URL,FORM,TYPE,SCHOOL,TIMEFRAME
0,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Venus and Adonis,1574-88,"Oil on canvas, 68 x 95 cm","Fogg Art Museum, Harvard University, Cambridge",https://www.wga.hu/art/a/aachen/adonis.jpg,painting,mythological,German,1601-1650
1,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)",Allegory,1598,"Oil on copper, 56 x 47 cm","Alte Pinakothek, Munich",https://www.wga.hu/art/a/aachen/allegory.jpg,painting,mythological,German,1601-1650
2,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)","Allegory of Peace, Art and Abundance",1602,"Oil on canvas, 197 x 142 cm","The Hermitage, St. Petersburg",https://www.wga.hu/art/a/aachen/allegorz.jpg,painting,mythological,German,1601-1650
3,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)","Jupiter, Antiope and Cupid",1595-98,"Oil on copper, 31 x 21 cm","Kunsthistorisches Museum, Vienna",https://www.wga.hu/art/a/aachen/antiope.jpg,painting,mythological,German,1601-1650
4,"AACHEN, Hans von","(b. 1552, Köln, d. 1615, Praha)","Pallas Athena, Venus and Juno",1593,"Oil on canvas, 54 x 67 cm","Museum of Fine Arts, Boston",https://www.wga.hu/art/a/aachen/athena.jpg,painting,mythological,German,1601-1650
...,...,...,...,...,...,...,...,...,...,...,...
144,"AERTSEN, Pieter","(b. 1508, Amsterdam, d. 1575, Amsterdam)",The Fat Kitchen. An Allegory,1565-75,"Oil on panel, 111 x 213 cm","Statens Museum for Kunst, Copenhagen",https://www.wga.hu/art/a/aertsen/fatkitch.jpg,painting,mythological,Netherlandish,1501-1550
145,"AERTSEN, Pieter","(b. 1508, Amsterdam, d. 1575, Amsterdam)",Market Scene,1561,"Oil on wood, 170 x 82,8 cm","Szépmûvészeti Múzeum, Budapest",https://www.wga.hu/art/a/aertsen/market.jpg,painting,genre,Netherlandish,1501-1550
146,"AERTSEN, Pieter","(b. 1508, Amsterdam, d. 1575, Amsterdam)",Market Scene (detail),1561,Oil on wood,"Szépmûvészeti Múzeum, Budapest",https://www.wga.hu/art/a/aertsen/market1.jpg,painting,genre,Netherlandish,1501-1550
147,"AERTSEN, Pieter","(b. 1508, Amsterdam, d. 1575, Amsterdam)",Market Scene,-,"Oil on oak, 127 x 85 cm","Wallraf-Richartz-Museum, Cologne",https://www.wga.hu/art/a/aertsen/marketsc.jpg,painting,genre,Netherlandish,1501-1550


In [ ]:
df.iloc[1] 

In [ ]:
df["SCHOOL"].nunique()

In [ ]:
df["AUTHOR"].nunique()

In [ ]:
df["TECHNIQUE"].unique()

In [ ]:
df["TYPE"].nunique()

In [ ]:
df["TIMEFRAME"].value_counts() 

In [ ]:
df_feature_encoded = df.drop(columns=["BORN-DIED", "TITLE", "DATE", "TECHNIQUE", "LOCATION", "URL", "FORM"])
df_feature_encoded

In [ ]:
transformer(df)

In [ ]:
df_col_encoded = get_col_transformed(transformer, df)
df_col_encoded

In [ ]:
img2 = np.array(df_col_encoded.iloc[2000:2001])
img2.shape

In [ ]:
df_col_encoded_reduced = df_col_encoded[:100]
df_col_encoded_reduced.shape

In [ ]:
knn_model = NearestNeighbors(n_neighbors=5).fit(df_col_encoded)
ind_list = list(knn_model.kneighbors(img2,n_neighbors=5)[1][0])#[1:] # get n closest points, unpack indices to a list
df_closest_images = df.iloc[ind_list, :]
df_closest_images

## Recommender System to KNN 

In [12]:
df_transformed.head()

,0,1,2,3,4,5,6,7,8,9,...,29990,29991,29992,29993,29994,29995,29996,29997,29998,29999
0,0.180392,0.168627,0.101961,0.262745,0.231373,0.180392,0.243137,0.196078,0.149020,0.184314,...,0.121569,0.149020,0.078431,0.062745,0.164706,0.098039,0.066667,0.149020,0.094118,0.058824
1,0.247059,0.239216,0.188235,0.180392,0.211765,0.160784,0.184314,0.231373,0.192157,0.172549,...,0.188235,0.403922,0.309804,0.207843,0.454902,0.364706,0.243137,0.435294,0.345098,0.223529
10,0.117647,0.086275,0.074510,0.760784,0.772549,0.705882,0.066667,0.050980,0.015686,0.070588,...,0.000000,0.086275,0.015686,0.070588,0.231373,0.149020,0.043137,0.133333,0.039216,0.031373
100,0.211765,0.207843,0.200000,0.219608,0.215686,0.207843,0.200000,0.188235,0.160784,0.223529,...,0.341176,0.419608,0.376471,0.368627,0.474510,0.419608,0.407843,0.431373,0.376471,0.364706
101,0.211765,0.164706,0.109804,0.196078,0.149020,0.094118,0.196078,0.149020,0.094118,0.207843,...,0.109804,0.113725,0.113725,0.105882,0.137255,0.137255,0.129412,0.235294,0.215686,0.192157


In [ ]:
cols_to_encode = ["TYPE", "SCHOOL"]
cat_columns_idx = [df.columns.get_loc(col) for col in cols_to_encode]
cat_columns_idx 

In [ ]:
pipe = build_pipe_for_categorical(df)
array_transformed = pd.DataFrame(pipe.fit_transform(df))
df_array_transformed 

In [13]:
df_transformed.index

Index(['0', '1', '10', '100', '101', '103', '104', '105', '106', '107', '11',
       '114', '115', '116', '117', '118', '119', '12', '120', '121', '122',
       '123', '124', '125', '126', '127', '128', '129', '13', '130', '131',
       '132', '133', '135', '137', '138', '139', '14', '140', '141', '142',
       '143', '144', '145', '146', '147', '148', '15', '16', '17', '18', '19',
       '2', '20', '21', '22', '23', '28', '3', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45',
       '46', '47', '5', '51', '52', '53', '54', '59', '6', '60', '61', '7',
       '8', '9', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99'],
      dtype='object')

In [ ]:
from os.path import join

def join_img_df_with_encoded_df(df_transformed, pipe): 
    df_transformed.index = [int(i) for i in df_transformed.index] # transform index from str to int
    
    pipe = build_pipe_for_categorical(df) # call ohe
    array_transformed = pd.DataFrame(pipe.fit_transform(df))
    
    join_images_db = df_transformed.join(df_array_transformed, lsuffix='_left', rsuffix='_right')
    return join_images_db

join_img_df_with_encoded_df(df_transformed, pipe)

In [ ]:
img_db = flat_images_db_download(size=100, source='gcp', rm=True)
img_db.index = [int(i) for i in img_db.index]
img_db

In [ ]:
pipe = build_pipe_for_categorical(get_data_locally(nrows=100)) # call ohe
array_transformed = pd.DataFrame(pipe.fit_transform(get_data_locally(nrows=100)))
join_images_db = img_db.join(array_transformed,  lsuffix='_left', rsuffix='_right')
join_images_db

In [41]:
create_joined_img_df(build_pipe_for_categorical, create_flat_images_db)

100%|██████████████████████████████| 100/100 [00:02<00:00, 38.10it/s]


,0_left,1_left,2_left,3_left,4_left,5_left,6_left,7_left,8_left,9_left,...,6_right,7_right,8_right,9_right,10_right,11_right,12_right,13_right,14_right,15_right
60,0.549020,0.521569,0.490196,0.521569,0.494118,0.462745,0.486275,0.466667,0.450980,0.396078,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
148,0.015686,0.011765,0.003922,0.019608,0.027451,0.023529,0.003922,0.011765,0.007843,0.023529,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,0.576471,0.627451,0.701961,0.572549,0.619608,0.713726,0.560784,0.623529,0.713726,0.556863,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
59,0.329412,0.274510,0.239216,0.325490,0.266667,0.254902,0.305882,0.247059,0.227451,0.305882,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,0.219608,0.196078,0.149020,0.329412,0.286275,0.215686,0.250980,0.298039,0.258824,0.388235,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,0.274510,0.227451,0.235294,0.462745,0.400000,0.349020,0.501961,0.439216,0.388235,0.482353,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
147,0.109804,0.090196,0.074510,0.125490,0.121569,0.113725,0.129412,0.109804,0.094118,0.117647,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,0.223529,0.203922,0.188235,0.152941,0.149020,0.141176,0.250980,0.231373,0.215686,0.211765,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
0,0.274510,0.282353,0.188235,0.215686,0.200000,0.164706,0.180392,0.168627,0.149020,0.317647,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [54]:
flat_images_db_upload(size=100, rm=False)


=> flat_resized_images_100.joblib uploaded to bucket vincent-van-bot-bucket inside data/flat_resized_images_100.joblib


### from title test 

In [49]:
from vincentvanbot.fromtitle.data_rec import create_joined_img_df, create_flat_images_db, get_data_locally
from vincentvanbot.fromtitle.predict_rec import search_title, user_input_transformed, get_closest_images_indexes
from vincentvanbot.fromtitle.train_rec import train_model

In [57]:
create_joined_img_df(build_pipe_for_categorical, create_flat_images_db)

100%|██████████████████████████████| 100/100 [00:02<00:00, 40.92it/s]


,0_left,1_left,2_left,3_left,4_left,5_left,6_left,7_left,8_left,9_left,...,6_right,7_right,8_right,9_right,10_right,11_right,12_right,13_right,14_right,15_right
60,0.549020,0.521569,0.490196,0.521569,0.494118,0.462745,0.486275,0.466667,0.450980,0.396078,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
148,0.015686,0.011765,0.003922,0.019608,0.027451,0.023529,0.003922,0.011765,0.007843,0.023529,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,0.576471,0.627451,0.701961,0.572549,0.619608,0.713726,0.560784,0.623529,0.713726,0.556863,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
59,0.329412,0.274510,0.239216,0.325490,0.266667,0.254902,0.305882,0.247059,0.227451,0.305882,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,0.219608,0.196078,0.149020,0.329412,0.286275,0.215686,0.250980,0.298039,0.258824,0.388235,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,0.274510,0.227451,0.235294,0.462745,0.400000,0.349020,0.501961,0.439216,0.388235,0.482353,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
147,0.109804,0.090196,0.074510,0.125490,0.121569,0.113725,0.129412,0.109804,0.094118,0.117647,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,0.223529,0.203922,0.188235,0.152941,0.149020,0.141176,0.250980,0.231373,0.215686,0.211765,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
0,0.274510,0.282353,0.188235,0.215686,0.200000,0.164706,0.180392,0.168627,0.149020,0.317647,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [55]:
flat_images_db_upload(size=100, rm=False)


=> flat_resized_images_100.joblib uploaded to bucket vincent-van-bot-bucket inside data/flat_resized_images_100.joblib


In [56]:
joblib.load("/Users/juliaschmitt/Downloads/data_flat_resized_images_100.joblib") 

,0,1,2,3,4,5,6,7,8,9,...,4526,4527,4528,4529,4530,4531,4532,4533,4534,4535
0,0.274510,0.282353,0.188235,0.215686,0.200000,0.164706,0.180392,0.168627,0.149020,0.317647,...,0.078431,0.125490,0.086275,0.078431,0.090196,0.062745,0.039216,0.152941,0.109804,0.094118
1,0.262745,0.278431,0.211765,0.254902,0.294118,0.250980,0.219608,0.290196,0.243137,0.160784,...,0.156863,0.364706,0.301961,0.203922,0.345098,0.294118,0.231373,0.305882,0.231373,0.164706
10,0.066667,0.054902,0.027451,0.058824,0.031373,0.007843,0.078431,0.050980,0.027451,0.094118,...,0.000000,0.286275,0.117647,0.050980,0.113725,0.035294,0.000000,0.074510,0.019608,0.019608
100,0.176471,0.172549,0.164706,0.164706,0.145098,0.121569,0.192157,0.152941,0.149020,0.141176,...,0.243137,0.360784,0.301961,0.274510,0.356863,0.294118,0.298039,0.286275,0.239216,0.239216
101,0.254902,0.192157,0.094118,0.239216,0.172549,0.101961,0.215686,0.145098,0.098039,0.254902,...,0.105882,0.133333,0.129412,0.121569,0.137255,0.137255,0.129412,0.113725,0.109804,0.094118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.298039,0.247059,0.211765,0.301961,0.274510,0.243137,0.356863,0.286275,0.239216,0.298039,...,0.505882,0.678431,0.592157,0.447059,0.894118,0.827451,0.725490,0.674510,0.607843,0.505882
96,0.423529,0.427451,0.447059,0.380392,0.364706,0.329412,0.450980,0.407843,0.384314,0.298039,...,0.325490,0.349020,0.305882,0.235294,0.407843,0.360784,0.305882,0.345098,0.298039,0.235294
97,0.192157,0.156863,0.160784,0.262745,0.235294,0.211765,0.286275,0.243137,0.227451,0.286275,...,0.501961,0.552941,0.517647,0.490196,0.525490,0.501961,0.439216,0.509804,0.498039,0.470588
98,0.196078,0.200000,0.207843,0.192157,0.227451,0.223529,0.223529,0.243137,0.258824,0.294118,...,0.282353,0.713726,0.721569,0.627451,0.670588,0.678431,0.584314,0.666667,0.682353,0.623529


### get_user input transformed

In [11]:
from vincentvanbot.utils import load_img, download_single_image


In [ ]:
def search_title(regex: str, df, case=False):
    """User input is title of image as a string. Search title column of original df, return respective row(s) with any matches."""
    matched_row = df[df['TITLE'].str.contains(regex, regex=True, case=case, na=False)][:1]
    return matched_row

search_title("art", df, case=False)  

In [ ]:
def download_and_preprocess_user_img():
    """Get local path (download), correlating index and apply preprocessing function. Returns flat version of image vector."""
    user_df = search_title(regex: str, df, case=False)
    download_single_image(user_df)
    path = "raw_data/images/..."
    img = load_img(path,target_size=(100,100), interpolation='nearest')
    user_input_transformed = preprocess_image(img, dim=(100,100))
    return user_input_transformed


In [ ]:
# input transformed version of user input to trained KNN
# catalog csv get url - download image locally or request library 


### PCA - check visually with different components

In [ ]:
lfw_path = "/Users/juliaschmitt/code/JulSchmitt/vincentvanbot/raw_data/images"

images = []
for root, dirnames, filenames in os.walk(lfw_path):
    for filename in fnmatch.filter(filenames, '*.jpg'):
        images.append(os.path.join(root, filename))

n = len(images)
print("loaded %d images" % n)

In [ ]:
# random selection of images

img = [Image.open(images[int(n*random.random())]) for i in range(8)]
#img[1] = np.concatenate([np.array(im.getdata()).reshape((im.size[0], im.size[1], 3)) for im in img ], axis=1)
#plt.figure(figsize=(16,2))
#plt.imshow(Image.fromarray(img.astype('uint8')))


#concat_image = np.concatenate([np.asarray(t) for t in img], axis=1)

#img = [[X[idx].reshape((w,h,3))] for idx in np.random.randint(n, size=8)]
#img = np.concatenate([np.asarray(t) for t in img], axis=1)
#plt.figure(figsize=(16,8))
#plt.imshow(Image.fromarray(img_all.astype('uint8')))

#img = img.resize((int(img.width * 100 / img.height), 100))

In [ ]:
n_components = 50
pca = PCA(n_components=n_components, svd_solver='randomized', whiten=True)
pca.fit(X)

In [ ]:
Xp = pca.transform(df)
Xp.shape

In [ ]:
Xt = pca.inverse_transform(Xp)
Xt = np.clip(Xt, 0, 255)

In [ ]:
w = h = 100

In [ ]:
test_1 = X[1].reshape((w,h,3)) #for idx in np.random.randint(n, size=8)][1]
test_2 = Xt[1].reshape((w,h,3)) #for idx in np.random.randint(n, size=8)][1]
stack_img = np.hstack((test_1, test_2))
plt.figure(figsize=(32,4))
plt.imshow(stack_img)


In [ ]:
test_x = [X[idx].reshape((w,h,3)) for idx in np.random.randint(n, size=8)][1]
plt.imshow(test_x)

In [ ]:
test_xt = [Xt[idx].reshape((w,h,3)) for idx in np.random.randint(n, size=8)][1]
plt.imshow(test_xt)

In [ ]:
npca = [100, 50, 10, 2, 1]
Xt = []
for p in npca:
    print("computing PCA for %d components"%p)
    pca = PCA(n_components=p, svd_solver='randomized', whiten=True)
    pca.fit(X)
    Xp = pca.transform(X)
    Xt.append(np.clip(pca.inverse_transform(Xp), 0, 255))

In [ ]:
idxs = np.random.randint(n, size=3)
for i,idx in enumerate(idxs):
    img_orig = X[idx].reshape((w,h,3))
    img_reconstructed = np.concatenate([Xt[p][idx].reshape((w,h,3)) for p in range(len(npca))], axis=1)
    plt.figure(figsize=(2,2))
    plt.imshow(Image.fromarray(img_orig.astype('uint8')))
    plt.figure(figsize=(16,2))
    plt.imshow(Image.fromarray(img_reconstructed.astype('uint8')))


In [ ]:
img_pairs = [[X[idx].reshape((w,h,3)), Xt[idx].reshape((w,h,3))] for idx in np.random.randint(n, size=8)]
img_all = np.concatenate([np.concatenate([p[0], p[1]]) for p in img_pairs], axis=1)
plt.figure(figsize=(32,8))
plt.imshow(Image.fromarray(img_all.astype('uint8')))
